In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os, sys
import numpy as np
from collections import OrderedDict as OD
import math
#import uproot3
import uproot as uproot
import hist
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import enum
import mplhep as hep

sys.path.insert(1, '../') # to import file from other directory (../ in this case)

from htoaa_Settings import *

#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230831_SelPNetMDXbbNSV/2018/analyze_htoaa_stage1.root'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230831_SelPNetMDXbbNSV/2018/plots'
#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230921_SFPNetMDXbbvsQCD/2018/analyze_htoaa_stage1.root'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230921_SFPNetMDXbbvsQCD/2018/plots'
sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230922_DataSplitByEra/2018/analyze_htoaa_stage1.root'
sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230922_DataSplitByEra/2018/plots'



if not os.path.exists(sOpDir):
    os.makedirs(sOpDir)
    
fIpFile = uproot.open(sIpFile)

In [3]:
fIpFile['evt/SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150'].keys()

['hPV_npvs_beforeSel_central;1',
 'hPV_npvsGood_beforeSel_central;1',
 'hGenLHE_HT_all_central;1',
 'hGenLHE_HTIncoming_all_central;1',
 'hGenLHE_Vpt_all_central;1',
 'hGenLHE_AlphaS_all_central;1',
 'hGenLHE_Njets_all_central;1',
 'hGenLHE_Nb_all_central;1',
 'hGenLHE_Nc_all_central;1',
 'hGenLHE_Nuds_all_central;1',
 'hGenLHE_Nglu_all_central;1',
 'hGenLHE_NpNLO_all_central;1',
 'hGenLHE_NpLO_all_central;1',
 'hPileup_nTrueInt_central;1',
 'hPileup_nPU_central;1',
 'hCutFlow_SR_central;1',
 'hCutFlowWeighted_SR_central;1',
 'hPV_npvs_SR_central;1',
 'hPV_npvsGood_SR_central;1',
 'nSelFatJet_SR_central;1',
 'hLeadingFatJetPt_SR_central;1',
 'hLeadingFatJetEta_SR_central;1',
 'hLeadingFatJetPhi_SR_central;1',
 'hLeadingFatJetMass_SR_central;1',
 'hLeadingFatJetMSoftDrop_SR_central;1',
 'hLeadingFatJetId_SR_central;1',
 'hLeadingFatJetPt_DataPreHEM1516Issue_SR_central;1',
 'hLeadingFatJetEta_DataPreHEM1516Issue_SR_central;1',
 'hLeadingFatJetPhi_DataPreHEM1516Issue_SR_central;1',
 'hLea

In [4]:
#h = fIpFile['evt/SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150/hLeadingFatJetMass_central']

In [5]:
fIpFile['evt'].keys()

['Data;1',
 'Data/hPV_npvs_beforeSel_noweight;1',
 'Data/hPV_npvsGood_beforeSel_noweight;1',
 'Data/hCutFlow_SR_noweight;1',
 'Data/hCutFlowWeighted_SR_noweight;1',
 'Data/hPV_npvs_SR_noweight;1',
 'Data/hPV_npvsGood_SR_noweight;1',
 'Data/nSelFatJet_SR_noweight;1',
 'Data/hLeadingFatJetPt_SR_noweight;1',
 'Data/hLeadingFatJetEta_SR_noweight;1',
 'Data/hLeadingFatJetPhi_SR_noweight;1',
 'Data/hLeadingFatJetMass_SR_noweight;1',
 'Data/hLeadingFatJetMSoftDrop_SR_noweight;1',
 'Data/hLeadingFatJetId_SR_noweight;1',
 'Data/hLeadingFatJetPt_DataPreHEM1516Issue_SR_noweight;1',
 'Data/hLeadingFatJetEta_DataPreHEM1516Issue_SR_noweight;1',
 'Data/hLeadingFatJetPhi_DataPreHEM1516Issue_SR_noweight;1',
 'Data/hLeadingFatJetPt_DataWithHEM1516Issue_SR_noweight;1',
 'Data/hLeadingFatJetEta_DataWithHEM1516Issue_SR_noweight;1',
 'Data/hLeadingFatJetPhi_DataWithHEM1516Issue_SR_noweight;1',
 'Data/hLeadingFatJetPt_HEM1516IssueEtaPhiCut_SR_noweight;1',
 'Data/hLeadingFatJetEta_HEM1516IssuePhiCut_SR_noweig

In [6]:
#fIpFile['evt/Data'].keys()

['hPV_npvs_beforeSel_noweight;1',
 'hPV_npvsGood_beforeSel_noweight;1',
 'hCutFlow_SR_noweight;1',
 'hCutFlowWeighted_SR_noweight;1',
 'hPV_npvs_SR_noweight;1',
 'hPV_npvsGood_SR_noweight;1',
 'nSelFatJet_SR_noweight;1',
 'hLeadingFatJetPt_SR_noweight;1',
 'hLeadingFatJetEta_SR_noweight;1',
 'hLeadingFatJetPhi_SR_noweight;1',
 'hLeadingFatJetMass_SR_noweight;1',
 'hLeadingFatJetMSoftDrop_SR_noweight;1',
 'hLeadingFatJetId_SR_noweight;1',
 'hLeadingFatJetPt_DataPreHEM1516Issue_SR_noweight;1',
 'hLeadingFatJetEta_DataPreHEM1516Issue_SR_noweight;1',
 'hLeadingFatJetPhi_DataPreHEM1516Issue_SR_noweight;1',
 'hLeadingFatJetPt_DataWithHEM1516Issue_SR_noweight;1',
 'hLeadingFatJetEta_DataWithHEM1516Issue_SR_noweight;1',
 'hLeadingFatJetPhi_DataWithHEM1516Issue_SR_noweight;1',
 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_SR_noweight;1',
 'hLeadingFatJetEta_HEM1516IssuePhiCut_SR_noweight;1',
 'hLeadingFatJetPhi_HEM1516IssueEtaCut_SR_noweight;1',
 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_DataPreHEM151

In [7]:
#fIpFile['evt/SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150/hLeadingFatJetNConstituents_sel_leadingFatJetParticleNetMD_XbbvsQCD_central'].to_hist()

In [8]:
#fIpFile['evt/SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150/hLeadingFatJetNConstituents_sel_leadingFatJetParticleNetMD_XbbvsQCD_central'].to_hist().values()


In [9]:
def rebinTH1(h1_, nRebins):
    #print(f"rebinTH1():: histogram type {type(h1_) = },  {isinstance(h1_, hist.Hist) = }  ")
    if not isinstance(h1_, hist.Hist):
        print(f"rebinTH1():: histogram type {type(h1_)} not implemented... so could not rebin histogram ")
        return h1_

    h1Rebin_ = None
    if   nRebins == 1:
        h1Rebin_ = h1_
    elif   nRebins == 2:
        h1Rebin_ = h1_[::2j]
    elif nRebins == 3:
        h1Rebin_ = h1_[::3j]
    elif nRebins == 4:
        h1Rebin_ = h1_[::4j]
    elif nRebins == 5:
        h1Rebin_ = h1_[::5j]
    elif nRebins == 6:
        h1Rebin_ = h1_[::6j]
    elif nRebins == 10:
        h1Rebin_ = h1_[::10j]
    elif nRebins == 20:
        h1Rebin_ = h1_[::20j]
    elif nRebins == 40:
        h1Rebin_ = h1_[::40j]
    elif nRebins == 50:
        h1Rebin_ = h1_[::50j]
    elif nRebins == 100:
        h1Rebin_ = h1_[::100j]
        print("Rebin 100 <<<")
    else:
        print(f"nRebins={nRebins} is not yet implemented... Implement it \t\t **** ERROR ****")        
        
    #print(f"h1_ values ({len(h1_.values())}): {h1_.values()} \n variances ({len(h1_.variances())}): {h1_.variances()}")
    #print(f"h1Rebin_ values ({len(h1Rebin_.values())}): {h1Rebin_.values()} \n variances ({len(h1Rebin_.variances())}): {h1Rebin_.variances()}")
    if   nRebins > 1:    
        h1_ = h1Rebin_

    return h1_

In [10]:
def getNonZeroMin(arr):
    min_ = 1e20
    a_   = arr[np.nonzero(arr)]
    if len(a_) > 0:
        min_ = np.min( a_ )
    return min_


In [11]:
# Function to draw box error bars
# https://matplotlib.org/stable/gallery/statistics/errorbars_and_boxes.html#sphx-glr-gallery-statistics-errorbars-and-boxes-py
def make_error_boxes(ax, xdata, ydata, xerror, yerror, facecolor='lightgrey',
                     edgecolor='none', alpha=0.5):

    # Loop over data points; create box from errors at each point
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.patches.Rectangle.html
    # matplotlib.patches.Rectangle(xy, width, height, *, angle=0.0, rotation_point='xy', **kwargs)
    #errorboxes = [Rectangle((x - xe[0], y - ye[0]), xe.sum(), ye.sum())
    #              for x, y, xe, ye in zip(xdata, ydata, xerror.T, yerror.T)]
    errorboxes = [Rectangle((x - xe, y - ye), 2*xe, 2*ye)
                  for x, y, xe, ye in zip(xdata, ydata, xerror.T, yerror.T)]

    # Create patch collection with specified colour/alpha
    pc = PatchCollection(errorboxes, facecolor=facecolor, alpha=alpha,
                         edgecolor=edgecolor)

    # Add collection to axes
    ax.add_collection(pc)

    artists = None
    # Plot errorbars
    #artists = ax.errorbar(xdata, ydata, xerr=xerror, yerr=yerror,
    #                      fmt='none', ecolor=facecolor)

    return artists

In [12]:
sXRange = "xAxisRange"; sYRange = "yAxisRange";
sXLabel = 'xAxisLabel'; sYLabel = 'yAxisLabel';
sNRebinX = 'nRebinX';  sNRebinY = 'nRebinY'; 

class DataBlindingOptions(enum.Enum):
    BlindPartially = '(partially blind)'
    BlindFully     = '(blind)'
    Unblind        = ' '

era = 2018
luminosity = 54.54  #59.83
cmsWorkStatus='Work in Progress'


dataBlindOption = DataBlindingOptions.Unblind # DataBlindingOptions.BlindPartially , DataBlindingOptions.BlindFully , DataBlindingOptions.Unblind
ExpData_dict = {
    'Data': ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D']
}
#MCBkg_list = ['QCD_bEnrich', 'QCD_bGen', 'QCD_Incl', 'TTJets_NLO', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = ['QCD_bEnrich', 'QCD_bGen', 'QCD_Incl', 'TTJets_HT', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = ['QCD_bEnrich', 'QCD_bGen', 'QCD_Incl', 'TTJets_Incl', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat',  'TTJets_NLO', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
MCBkg_list = ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat',  'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = ['QCD_0bCat', 'QCD_1bCat']
MCSig_list = ['SUSY_GluGluH_01J_HToAATo4B_M-20_HPtAbv150', 'SUSY_GluGluH_01J_HToAATo4B_M-30_HPtAbv150', ]
sLableSig = ['HToAATo4B_M-20', 'HToAATo4B_M-30']
systematics_list = ['central']
systematics_forData = 'noweight'
selectionTags = ['sel_leadingFatJetParticleNetMD_XbbvsQCD'] # ['sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_XbbvsQCD', 'SR'] #['SR', 'sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_XbbvsQCD', 'sel_2018HEM1516Issue']

sData = 'Data' # "Data"
scale_MCSig = 50 #1000
yRatioLimit = [0.4, 1.6]

logYMinScaleFactor = 100 # 1 # scale yMin by factor logYMinScaleFactor to not concentrate lowest stats background processes

significantThshForDataBlinding = 0.125 # blind data in bins with S/sqrt(B) > significantThshForDataBlinding while running with dataBlindOption = DataBlindingOptions.BlindPartially


histograms_dict = OD([
    #("hLeadingFatJetMass", {sXLabel: 'Leading FatJet mass [GeV]', sYLabel: 'Events', sXRange: [50, 250], sYRange: [1e-2, 1e8]})
    #("hLeadingFatJetMass", {sXLabel: 'Leading FatJet mass [GeV]', sYLabel: 'Events', sXRange: [50, 250]}),
    
    #("", {sXLabel: '', sYLabel: 'Events', sXRange: []}),
    
    #("hCutFlow", {sXLabel: 'hCutFlow', sYLabel: 'Events'}),
    #("hCutFlowWeighted", {sXLabel: 'hCutFlowWeighted', sYLabel: 'Events'}),

    #("hPV_npvs_beforeSel", {sXLabel: 'No. of primary vertices before selection', sYLabel: 'Events', sXRange: [0, 100] }),
    #("hPV_npvsGood_beforeSel", {sXLabel: 'No. of good primary vertices before selection', sYLabel: 'Events', sXRange: [0, 100] }),
    #("hPV_npvs_SR", {sXLabel: 'No. of primary vertices in SR', sYLabel: 'Events', sXRange: [0, 100] }),
    #("hPV_npvsGood_SR", {sXLabel: 'No. of good primary vertices in SR', sYLabel: 'Events', sXRange: [0, 100] }),

    ("hLeadingFatJetPt", {sXLabel: 'hLeadingFatJetPt', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    ("hLeadingFatJetEta", {sXLabel: 'hLeadingFatJetEta', sYLabel: 'Events', sXRange: [-3.5, 3.5], sNRebinX: 2 }),
    ("hLeadingFatJetPhi", {sXLabel: 'hLeadingFatJetPhi', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),

    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),
    #("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    #("hLeadingFatJetEta_HEM1516IssuePhiCut_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    #("hLeadingFatJetPhi_HEM1516IssueEtaCut_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),
    #("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    #("hLeadingFatJetEta_HEM1516IssuePhiCut_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    #("hLeadingFatJetPhi_HEM1516IssueEtaCut_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),

    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516Fix", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516Fix', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4  }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516Fix", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516Fix', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516Fix", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516Fix', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),
    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516Fix_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516Fix_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4  }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516Fix_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516Fix_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516Fix_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516Fix_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),
    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516Fix_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516Fix_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516Fix_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516Fix_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516Fix_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516Fix_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),

    # 2018 HEM15/16 issue validation
    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516MCRewgt", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516MCRewgt', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516MCRewgt", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516MCRewgt', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516MCRewgt", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516MCRewgt', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),
    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516MCRewgt_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516MCRewgt_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516MCRewgt_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516MCRewgt_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516MCRewgt_DataPreHEM1516Issue", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516MCRewgt_DataPreHEM1516Issue', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),
    ("hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516MCRewgt_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_woHEM1516MCRewgt_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 4 }),
    ("hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516MCRewgt_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetEta_HEM1516IssuePhiCut_woHEM1516MCRewgt_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [-3.5, 3.5] }),
    ("hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516MCRewgt_DataWithHEM1516Issue", {sXLabel: 'hLeadingFatJetPhi_HEM1516IssueEtaCut_woHEM1516MCRewgt_DataWithHEM1516Issue', sYLabel: 'Events', sXRange: [-3.14, 3.14], sNRebinX: 2 }),

    ("hLeadingFatJetMass", {sXLabel: 'hLeadingFatJetMass', sYLabel: 'Events', sXRange: [50, 300], sNRebinX: 5}),
    ("hLeadingFatJetMSoftDrop", {sXLabel: 'hLeadingFatJetMSoftDrop', sYLabel: 'Events', sXRange: [50, 300], sNRebinX: 5 }),
    ("hLeadingFatJetBtagDeepB", {sXLabel: 'hLeadingFatJetBtagDeepB', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetBtagDDBvLV2", {sXLabel: 'hLeadingFatJetBtagDDBvLV2', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetBtagDDCvBV2", {sXLabel: 'hLeadingFatJetBtagDDCvBV2', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    
    ("hLeadingFatJetBtagHbb", {sXLabel: 'hLeadingFatJetBtagHbb', sYLabel: 'Events', sXRange: [-1, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_H4qvsQCD", {sXLabel: 'hLeadingFatJetDeepTagMD_H4qvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_HbbvsQCD", {sXLabel: 'hLeadingFatJetDeepTagMD_HbbvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_ZHbbvsQCD", {sXLabel: 'hLeadingFatJetDeepTagMD_ZHbbvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_ZHccvsQCD", {sXLabel: 'hLeadingFatJetDeepTagMD_ZHccvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    
    ("hLeadingFatJetDeepTagMD_ZbbvsQCD", {sXLabel: 'hLeadingFatJetDeepTagMD_ZbbvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_ZvsQCD", {sXLabel: 'hLeadingFatJetDeepTagMD_ZvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_bbvsLight", {sXLabel: 'hLeadingFatJetDeepTagMD_bbvsLight', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTagMD_ccvsLight", {sXLabel: 'hLeadingFatJetDeepTagMD_ccvsLight', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTag_H", {sXLabel: 'hLeadingFatJetDeepTag_H', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    
    ("hLeadingFatJetDeepTag_QCD", {sXLabel: 'hLeadingFatJetDeepTag_QCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetDeepTag_QCDothers", {sXLabel: 'hLeadingFatJetDeepTag_QCDothers', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetN2b1", {sXLabel: 'hLeadingFatJetN2b1', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetN3b1", {sXLabel: 'hLeadingFatJetN3b1', sYLabel: 'Events', sXRange: [0, 5], sNRebinX: 2 }),
    ("hLeadingFatJetTau1", {sXLabel: 'hLeadingFatJetTau1', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    
    ("hLeadingFatJetTau2", {sXLabel: 'hLeadingFatJetTau2', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetTau3", {sXLabel: 'hLeadingFatJetTau3', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetTau4", {sXLabel: 'hLeadingFatJetTau4', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetTau4by3", {sXLabel: 'hLeadingFatJetTau4by3', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetTau3by2", {sXLabel: 'hLeadingFatJetTau3by2', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    
    ("hLeadingFatJetTau2by1", {sXLabel: 'hLeadingFatJetTau2by1', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetNConstituents", {sXLabel: 'hLeadingFatJetNConstituents', sYLabel: 'Events'}),
    ("hLeadingFatJetParticleNetMD_QCD", {sXLabel: 'hLeadingFatJetParticleNetMD_QCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetParticleNetMD_Xbb", {sXLabel: 'hLeadingFatJetParticleNetMD_Xbb', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetParticleNetMD_Xcc", {sXLabel: 'hLeadingFatJetParticleNetMD_Xcc', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),       
    ("hLeadingFatJetParticleNetMD_Xqq", {sXLabel: 'hLeadingFatJetParticleNetMD_Xqq', sYLabel: 'Events', sNRebinX: 2}),

    ("hLeadingFatJetParticleNetMD_XbbOverQCD", {sXLabel: 'hLeadingFatJetParticleNetMD_XbbOverQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }), 
    ("hLeadingFatJetParticleNetMD_XccOverQCD", {sXLabel: 'hLeadingFatJetParticleNetMD_XccOverQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }), 
    ("hLeadingFatJetParticleNetMD_XqqOverQCD", {sXLabel: 'hLeadingFatJetParticleNetMD_XqqOverQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }), 

    ("hLeadingFatJetParticleNet_H4qvsQCD", {sXLabel: 'hLeadingFatJetParticleNet_H4qvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetParticleNet_HbbvsQCD", {sXLabel: 'hLeadingFatJetParticleNet_HbbvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetParticleNet_HccvsQCD", {sXLabel: 'hLeadingFatJetParticleNet_HccvsQCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    ("hLeadingFatJetParticleNet_QCD", {sXLabel: 'hLeadingFatJetParticleNet_QCD', sYLabel: 'Events', sXRange: [0, 1], sNRebinX: 2 }),
    
    ("hLeadingFatJetParticleNet_mass", {sXLabel: 'hLeadingFatJetParticleNet_mass', sYLabel: 'Events', sXRange: [50, 300], sNRebinX: 5 }),

    ("hLeadingFatJet_nSubJets", {sXLabel: 'hLeadingFatJet_nSubJets', sYLabel: 'Events', sXRange: [-0.5, 10.5] }),
    ("hLeadingFatJet_nSubJets_bTag_L", {sXLabel: 'hLeadingFatJet_nSubJets_bTag_L', sYLabel: 'Events', sXRange: [-0.5, 10.5] }),
    ("hLeadingFatJet_nSubJets_bTag_M", {sXLabel: 'hLeadingFatJet_nSubJets_bTag_M', sYLabel: 'Events', sXRange: [-0.5, 10.5] }),
    ("hLeadingFatJet_nSV", {sXLabel: 'hLeadingFatJet_nSV', sYLabel: 'Events', sXRange: [-0.5, 10.5] }),

    ("hMET_pT", {sXLabel: 'hMET_pT', sYLabel: 'Events', sXRange: [0, 1000], sNRebinX: 5 }),
    ("hMET_sumEt", {sXLabel: 'hMET_sumEt', sYLabel: 'Events', sXRange: [0, 4000], sNRebinX: 5 }),

    ("hLeadingFatJet_nLeptons", {sXLabel: 'hLeadingFatJet_nLeptons', sYLabel: 'Events', sXRange: [-0.5, 10.5] }),

    #("", {sXLabel: '', sYLabel: 'Events'}),
])

'''
histograms_dict = OD([
    ("hLeadingFatJetPt", {sXLabel: 'hLeadingFatJetPt', sYLabel: 'Events', sXRange: [180, 1000], sNRebinX: 2 }),
    #("hLeadingFatJetPt", {sXLabel: 'hLeadingFatJetPt', sYLabel: 'Events', sXRange: [800, 900], sYRange: [100, 1000] }),
])
'''

#colors_bkg_list = ['blue', 'orange', 'brown'] # ["#9b59b6", "#e74c3c", "#34495e", "#2ecc71"] #['lightcoral', 'burlywood', 'cyan', 'saddlebrown', 'slateblue', 'lightpink', 'darkkhaki', 'antiquewhite', 'limegreen', 'violet', 'firebrick', 'darkorchid', 'tan', 'olive', 'purple']

colors_bkg_list = [ 
    # ['color', <transperent>, '<fill pattern>']
    ['lightcoral', 0.9, ''],
    ['cyan', 0.9, '' ],
    ['burlywood', 0.9, '' ], 
    ['saddlebrown', 0.9, '' ],
    ['slateblue', 0.9, '' ],
    ['lightpink', 0.9, 'xx' ],
    ['darkkhaki', 0.9, '' ],
    ['antiquewhite', 0.9, '//' ],
    ['limegreen', 0.9, '' ],
    ['violet', 0.9, '' ],
    ['firebrick', 0.9, '' ],
    ['darkorchid', 0.9, '' ],
    ['tan', 0.9, '' ],
    ['olive', 0.9, '' ],
    ['purple',  0.9, ''],
]

colors_sig_list = [
    # ['color', <transperent>, '<fill pattern>', ]
    ['blue', 0.9, ''],
    ['red', 0.9, ''],
    ['green', 0.9, ''],
]

for selectionTag in selectionTags:    
    dataBlindOption_toUse = dataBlindOption if selectionTag != 'SR' else DataBlindingOptions.BlindPartially
    for histo_name in histograms_dict.keys():
        histo_name_toUse = '%s_%s' % (histo_name, selectionTag)
        for systematic in systematics_list:
            for yAxisScale in ['linearY', 'logY']: # ['linearY', 'logY']
                xAxisRange = histograms_dict[histo_name][sXRange] if sXRange in histograms_dict[histo_name].keys() else None
                yAxisRange = histograms_dict[histo_name][sYRange] if sYRange in histograms_dict[histo_name].keys() else None
                xAxisLabel = histograms_dict[histo_name][sXLabel] if sXLabel in histograms_dict[histo_name].keys() else None
                yAxisLabel = histograms_dict[histo_name][sYLabel] if sYLabel in histograms_dict[histo_name].keys() else None
                nRebinX    = histograms_dict[histo_name][sNRebinX] if sNRebinX in histograms_dict[histo_name].keys() else 1

                yAxisRange_cal      = [1e20, -1e10]
                yRatioAxisRange_cal = [1e20, -1e10]
                xError = np.array([])
                hData = None
                hBkgTot_values = None
                hBkgTot_variance = None
                hStack_values_list = np.array([]) 
                hStack_edges = np.array([])
                hStack_centers = np.array([])
                sStack_list = []
                nBkgTot = 0
                significanceAvg = [] #np.array([])



                print(f"\n\n {histo_name_toUse = }, {systematic = }, {yAxisScale = }, ")
                #fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(8,10), sharex='col', gridspec_kw={'height_ratios': [3, 1]}, subplot_kw={'ymargin': 0.4})
                fig, ax = plt.subplots(ncols=1, nrows=2, figsize=(8,10), sharex='col', gridspec_kw={'height_ratios': [3, 1], 'hspace': 0})
                #print(f"fig: {fig}, axs: {axs}")

                #fig1, ax1 = plt.subplots()
                
                
                if len(MCBkg_list) > 0:
                    hBkg_list = []
                    sBkg_list = []
                    hBkg_integral_list = []
                    for dataset in MCBkg_list:
                        histo_name_toUse_full = 'evt/%s/%s_%s' % (dataset, histo_name_toUse, systematic)
                        h = fIpFile[histo_name_toUse_full].to_hist()
                        h = rebinTH1(h, nRebinX)

                        nTot_ = h.values().sum()
                        hBkg_list.append(h)
                        sBkg_list.append(dataset)
                        hBkg_integral_list.append(nTot_)

                        if abs(nTot_ - 0) < 1e-10: continue
                        #print(f"{histo_name_toUse_full} integral: {h.values.sum()}")

                        yMin_ = getNonZeroMin(h.values())
                        yMax_ = np.max(h.values())
                        if yMin_ < yAxisRange_cal[0]:
                            yAxisRange_cal[0] = yMin_
                        if yMax_ > yAxisRange_cal[1]:
                            yAxisRange_cal[1] = yMax_                        


                    # sort histograms in decreasing yield
                    isReverseSortForStack = True
                    idx_hBkg_sortedByIntegral = sorted(range(len(hBkg_integral_list)), key=lambda i: hBkg_integral_list[i], reverse=isReverseSortForStack)            

                    print(f"sBkg_list: {sBkg_list} \nhBkg_integral_list (total {sum(hBkg_integral_list)}): {hBkg_integral_list} ")
                    print(f"sBkg_list sorted by integral: {[(sBkg_list[idx], hBkg_integral_list[idx]) for idx in idx_hBkg_sortedByIntegral]}")            

                    hStack_list = [ hBkg_list[idx] for idx in idx_hBkg_sortedByIntegral ]  
                    sStack_list = [ sBkg_list[idx] for idx in idx_hBkg_sortedByIntegral ]  
                    print(f"sStack_list: {sStack_list}")
                    print(f"xError ({type(xError)}) ({len(xError)}): {xError}")

                    hStack_values_list    = np.array( [ h.values() for h in hStack_list ] )
                    hStack_variance_list  = np.array( [ h.variances() for h in hStack_list ] )
                    hStack_error_list     = np.array( [ np.sqrt(h.variances()) for h in hStack_list ] )
                    hStack_edges          = hStack_list[0].axes[0].edges
                    hStack_centers        = hStack_list[0].axes[0].centers
                    xError                = (hStack_list[0].axes[0].edges[1:] - hStack_list[0].axes[0].edges[0:-1]) / 2 if len(xError) == 0 else xError

                    hBkgTot_values        = np.sum(hStack_values_list, axis=0)
                    hBkgTot_variance      = np.sum(hStack_variance_list, axis=0)
                    print(f"hBkgTot_values sum: {sum(hBkgTot_values)}")

                    # Update yRange for hStackBkg -------
                    yMin_ = getNonZeroMin(hBkgTot_values)
                    yMax_ = np.max(hBkgTot_values)
                    if yMin_ < yAxisRange_cal[0]:
                        yAxisRange_cal[0] = yMin_
                    if yMax_ > yAxisRange_cal[1]:
                        yAxisRange_cal[1] = yMax_    

                    nHists = len(MCBkg_list)
                    colors_toUse = [ colors_bkg_list[i][0] for i in range(nHists) ]
                    alpha_toUse  = [ colors_bkg_list[i][1] for i in range(nHists) ]
                    hatch_toUse  = [ colors_bkg_list[i][2] for i in range(nHists) ]

                    # plot stack
                    #hep.histplot(hStack_values_list, bins=hStack_edges, ax=ax[0], yerr=hStack_error_list, histtype='fill', stack=True, label=sStack_list, sort='yield')
                    #hep.histplot(hStack_values_list, bins=hStack_edges, ax=ax[0], yerr=hStack_error_list, histtype='step', stack=True, label=sStack_list, linestyle="--", color=["green", "black", (1, 0, 0, 0.4)])
                    #hep.histplot(hStack_values_list, bins=hStack_edges, ax=ax[0], histtype='fill', stack=True, label=sStack_list, color=["green", "darkorange", "red"])

                    # https://matplotlib.org/stable/gallery/shapes_and_collections/hatch_style_reference.html
                    
                    hep.histplot(
                        hStack_values_list, 
                        bins=hStack_edges, 
                        ax=ax[0], 
                        histtype='fill', 
                        stack=True, 
                        label=sStack_list, 
                        color=colors_toUse,
                        alpha=alpha_toUse,
                        hatch=hatch_toUse,
                        sort='yield'
                        )

                    # plot total background
                    #hep.histplot(hBkgTot_values, bins=hStack_edges, ax=ax, yerr=np.sqrt(hBkgTot_variance), histtype='errorbar', color='grey', label='Total background')
                    
                    # plot totoal background error bars only
                    make_error_boxes(
                        ax=ax[0], 
                        xdata=hStack_centers, 
                        ydata=hBkgTot_values, 
                        xerror=xError, 
                        yerror=np.sqrt(hBkgTot_variance), 
                        facecolor='grey',
                        edgecolor='none', 
                        alpha=0.5
                        )
                    
                    # No. of events in total background
                    nBkgTot = np.sum(hBkgTot_values)


                    '''
                    if 1==1:
                        if xAxisRange: ax[0].set_xlim(xAxisRange[0], xAxisRange[1])
                        if yAxisRange: ax[0].set_ylim(yAxisRange[0], yAxisRange[1])
                        else:          ax[0].set_ylim(yAxisRange_cal[0], yAxisRange_cal[1])
                        if xAxisLabel: ax[0].set_xlabel(xAxisLabel)
                        if yAxisLabel: ax[0].set_ylabel(yAxisLabel)
                    '''
                print(f"\nAfter MCBkg {yAxisRange_cal = }")

                if len(MCSig_list) > 0:
                    hSig_list = []
                    sSig_list = []
                    hSig_integral_list = []
                    for iSig, dataset in enumerate(MCSig_list):
                        histo_name_toUse_full = 'evt/%s/%s_%s' % (dataset, histo_name_toUse, systematic)
                        h = fIpFile[histo_name_toUse_full].to_hist()
                        h = rebinTH1(h, nRebinX)

                        nTot_ = h.values().sum()
                        hSig_list.append(h)
                        sSig_list.append(dataset)
                        hSig_integral_list.append(h.values().sum())
                        print(f"{histo_name_toUse_full} integral: {h.values().sum()}")

                        histo_edges = h.axes[0].edges
                        xError      = (h.axes[0].edges[1:] - h.axes[0].edges[0:-1]) / 2 if len(xError) == 0 else xError


                        label_MCSig = dataset
                        label_MCSig = sLableSig[iSig]
                        if scale_MCSig > 1:
                            #label_MCSig = '%s x %d' % (dataset, scale_MCSig)
                            label_MCSig = '%s x %d' % (label_MCSig, scale_MCSig)
                            
                        yMin_ = getNonZeroMin(h.values())
                        yMax_ = np.max(h.values())
                        if yMin_ < yAxisRange_cal[0]:
                            yAxisRange_cal[0] = yMin_
                        if yMax_ > yAxisRange_cal[1]:
                            yAxisRange_cal[1] = yMax_                        

                        # plot signal
                        hep.histplot(
                            h.values() * scale_MCSig, 
                            bins=histo_edges, 
                            ax=ax[0], 
                            yerr=np.sqrt(h.variances()) * scale_MCSig, 
                            histtype='errorbar', 
                            label=label_MCSig,
                            color=colors_sig_list[iSig][0],                             
                            marker='o',
                            markerfacecolor=colors_sig_list[iSig][0],
                            markersize=3
                            )

                        nSig = np.sum(h.values())
                        # S/sqrt(B)
                        if nSig > 0 and nBkgTot > 0:
                            S_ = h.values() / nSig
                            B_ = np.sqrt(hBkgTot_values / nBkgTot)
                            significance_i = np.divide(S_, B_, where=B_!=0, out=np.zeros(len(B_)))
                            significanceAvg.append(significance_i)

                            #ax1.plot(h.axes[0].centers, significance_i)

                    significanceAvg = np.array(significanceAvg)
                    significanceAvg = np.sum(significanceAvg, axis=0)
                    significanceAvg = np.divide(significanceAvg, len(MCSig_list) )
                    print(f"significanceAvg (max: {np.max(significanceAvg)}): {significanceAvg}")




                print(f"\nAfter MCSig {yAxisRange_cal = }")
                
                if dataBlindOption_toUse in [DataBlindingOptions.Unblind, DataBlindingOptions.BlindPartially]: #sData:
                    histo_name_toUse_full = 'evt/%s/%s_%s' % (sData, histo_name_toUse, systematics_forData)
                    hData = fIpFile[histo_name_toUse_full].to_hist()
                    hData = rebinTH1(hData, nRebinX)
                    xError = (hData.axes[0].edges[1:] - hData.axes[0].edges[0:-1]) / 2

                    yMin_ = getNonZeroMin(hData.values())
                    yMax_ = np.max(hData.values())
                    if yMin_ < yAxisRange_cal[0]:
                        yAxisRange_cal[0] = yMin_
                    if yMax_ > yAxisRange_cal[1]:
                        yAxisRange_cal[1] = yMax_
                    #print(f"Data: {yMin_ = }, {yMin_}")

                    hData_values_toUse = hData.values()
                    hData_errors_toUse = np.sqrt(hData.variances())

                    # blind data with high S/sqrt(B) bins
                    #print(f"{len(significanceAvg) = }")
                    if dataBlindOption_toUse in [DataBlindingOptions.BlindPartially] and \
                        len(significanceAvg):
                        # inflate significantThshForDataBlinding for higher S/sqrt(B) when histogram is rebinned, 
                        # so that blinding of data is independent of rebinning
                        significantThshForDataBlinding_toUse = significantThshForDataBlinding * math.sqrt(nRebinX)
                        hData_values_toUse = np.where(
                            (significanceAvg > significantThshForDataBlinding_toUse),
                            np.full(len(hData_values_toUse), 0),
                            hData_values_toUse
                        )
                        hData_errors_toUse = np.where(
                            (significanceAvg > significantThshForDataBlinding_toUse),
                            np.full(len(hData_values_toUse), 0),
                            hData_errors_toUse
                        )
                        #print(f"{(significanceAvg > significantThshForDataBlinding) =}")
                        #print(f"hData_values_toUse ({len(hData_values_toUse)}): {hData_values_toUse}")


                    #hep.histplot(hData.values(), bins=hData.axes[0].edges, ax=ax[0], yerr=np.sqrt(hData.variances()), histtype='errorbar', color='black', label='Data')
                    hep.histplot(
                        hData_values_toUse, 
                        bins=hData.axes[0].edges, 
                        ax=ax[0], 
                        yerr=hData_errors_toUse, 
                        histtype='errorbar', 
                        color='black', 
                        label='%s %s' % ('Data', dataBlindOption_toUse.value)
                        )
                                        

                    #print(f"hData integral: {hData.values().sum()}")



                    # Ratio plot ---------------------------------------------------------                
                    ratio_values = np.divide(hData_values_toUse, hBkgTot_values, where=hBkgTot_values!=0, out=np.ones(len(hData.values())))
                    ratio_error  = hData_errors_toUse            
                    ratio_error  = np.divide(ratio_error, hBkgTot_values, where=hBkgTot_values!=0, out=np.zeros(len(hData.values())))
                    ratio_syst   = np.sqrt(hBkgTot_variance)
                    ratio_syst   = np.divide(ratio_syst, hBkgTot_values, where=hBkgTot_values!=0, out=np.zeros(len(hData.values())))

                    yMin_ = getNonZeroMin( ratio_values - ratio_error)
                    yMax_ = np.max( ratio_values + ratio_error)
                    if yMin_ < yRatioAxisRange_cal[0]:
                        yRatioAxisRange_cal[0] = yMin_
                    if yMax_ > yRatioAxisRange_cal[1]:
                        yRatioAxisRange_cal[1] = yMax_                          
                    
                    hep.histplot(
                        ratio_values, 
                        bins=hData.axes[0].edges, 
                        ax=ax[1], 
                        yerr=ratio_error, 
                        histtype='errorbar', 
                        color='black', 
                        label='Data'
                        )
                    #if xAxisRange: ax[1].set_xlim(xAxisRange[0], xAxisRange[1])

                    # plot totoal background error bars only for ratio plot
                    make_error_boxes(
                        ax=ax[1], 
                        xdata=hData.axes[0].centers, 
                        ydata=np.full(len(hData.axes[0].centers), 1), 
                        xerror=xError, 
                        yerror=ratio_syst, 
                        facecolor='grey',
                        edgecolor='none', 
                        alpha=0.5
                        )



                    
                # Upper plot cosmetics ---------
                if xAxisRange: ax[0].set_xlim(xAxisRange[0], xAxisRange[1])
                print(f"\nAt the end {yAxisRange_cal = }")
                if yAxisRange: ax[0].set_ylim(yAxisRange[0], yAxisRange[1])
                else:          
                    yMaxOffset = 10**(math.log10(yAxisRange_cal[1] / abs(yAxisRange_cal[0])) * 0.4) if yAxisScale == 'logY' else 1.6
                    #print(f"{yMaxOffset = }, {yAxisRange_cal[1] * yMaxOffset = }, \t\t {abs(yAxisRange_cal[0]) * logYMinScaleFactor = }")
                    if yAxisScale == 'logY':
                        yAxisRange_cal[0] = abs(yAxisRange_cal[0]) * logYMinScaleFactor
                        yAxisRange_cal[1] = yAxisRange_cal[1] * yMaxOffset
                    else:
                        yAxisRange_cal[0] = yAxisRange_cal[0]
                        yAxisRange_cal[1] = yAxisRange_cal[1] * yMaxOffset
                    print(f"\nAt the end updated {yAxisRange_cal = } \t {yAxisScale = }")
                    ax[0].set_ylim(yAxisRange_cal[0], yAxisRange_cal[1])
                if xAxisLabel: ax[0].set_xlabel(xAxisLabel)
                if yAxisLabel: ax[0].set_ylabel(yAxisLabel)                
                ax[0].legend(fontsize=12, loc='upper right', ncol=2)
                '''
                ax[0].legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncols=2, mode="expand", borderaxespad=0.)
                '''
                if yAxisScale == 'logY': ax[0].set_yscale('log', base=10)
                #ax[0].set_ymargin(1.)

                # Ratio plot cosmetics ---------
                if yRatioAxisRange_cal[0] < yRatioLimit[0]: yRatioAxisRange_cal[0] = yRatioLimit[0]
                if yRatioAxisRange_cal[1] > yRatioLimit[1]: yRatioAxisRange_cal[1] = yRatioLimit[1]
                yRatioAxisRange_cal_maxDeviation = max(abs(yRatioAxisRange_cal[0] - 1), abs(yRatioAxisRange_cal[1] - 1))
                yRatioAxisRange_cal[0] = 1 - yRatioAxisRange_cal_maxDeviation
                yRatioAxisRange_cal[1] = 1 + yRatioAxisRange_cal_maxDeviation
                if xAxisRange: ax[1].set_xlim(xAxisRange[0], xAxisRange[1]) 
                ax[1].set_ylim(yRatioAxisRange_cal[0], yRatioAxisRange_cal[1]) 

                if xAxisLabel: ax[1].set_xlabel(xAxisLabel)
                ax[1].set_ylabel('Data/MC')
                
                ax[1].axhline(y=1, linestyle='--')
                ax[1].grid()

                isData = True if dataBlindOption_toUse != DataBlindingOptions.BlindFully else False
                fontsize_toUse = 18 if isData else 15
                hep.cms.label(ax=ax[0], data=isData, year=era, lumi=luminosity, label=cmsWorkStatus, fontsize=fontsize_toUse)

                ax[0].text(0.6, 0.63, selectionTag,
                           fontsize=12, fontstyle='italic',
                            horizontalalignment='center',
                            verticalalignment='center',
                            transform=ax[0].transAxes
                            )

                fig.savefig('%s/%s_%s_%s.png' % (sOpDir,histo_name_toUse,systematic, yAxisScale), transparent=False, dpi=80, bbox_inches="tight")

                plt.close(fig)



 histo_name_toUse = 'hLeadingFatJetPt_sel_leadingFatJetMSoftDrop', systematic = 'central', yAxisScale = 'linearY', 
sBkg_list: ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat', 'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT'] 
hBkg_integral_list (total 26186364.26111905): [23712830.730377514, 589054.0393835319, 1244316.0772133637, 18622.61865580413, 11947.611801853576, 124.38941191511482, 158671.11335629647, 108143.26217035594, 12648.690554456964, 222412.33432527506, 107593.39386867693] 
sBkg_list sorted by integral: [('QCD_0bCat', 23712830.730377514), ('QCD_2bCat', 1244316.0772133637), ('QCD_1bCat', 589054.0393835319), ('WJetsToQQ_HT', 222412.33432527506), ('TTToHadronic_powheg', 158671.11335629647), ('TTToSemiLeptonic_powheg', 108143.26217035594), ('ZJetsToQQ_HT', 107593.39386867693), ('QCD_3bCat', 18622.61865580413), ('TTTo2L2Nu_powheg', 12648.690554456964), ('QCD_4bCat', 11947.611801853576), 



 histo_name_toUse = 'hLeadingFatJetPt_sel_leadingFatJetMSoftDrop', systematic = 'central', yAxisScale = 'logY', 
sBkg_list: ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat', 'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT'] 
hBkg_integral_list (total 26186364.26111905): [23712830.730377514, 589054.0393835319, 1244316.0772133637, 18622.61865580413, 11947.611801853576, 124.38941191511482, 158671.11335629647, 108143.26217035594, 12648.690554456964, 222412.33432527506, 107593.39386867693] 
sBkg_list sorted by integral: [('QCD_0bCat', 23712830.730377514), ('QCD_2bCat', 1244316.0772133637), ('QCD_1bCat', 589054.0393835319), ('WJetsToQQ_HT', 222412.33432527506), ('TTToHadronic_powheg', 158671.11335629647), ('TTToSemiLeptonic_powheg', 108143.26217035594), ('ZJetsToQQ_HT', 107593.39386867693), ('QCD_3bCat', 18622.61865580413), ('TTTo2L2Nu_powheg', 12648.690554456964), ('QCD_4bCat', 11947.611801853576), ('Q

/tmp/ssawant/ipykernel_27712/2615395917.py:519: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax[1].set_ylim(yRatioAxisRange_cal[0], yRatioAxisRange_cal[1])




 histo_name_toUse = 'hLeadingFatJetPt_HEM1516IssueEtaPhiCut_DataWithHEM1516Issue_sel_leadingFatJetMSoftDrop', systematic = 'central', yAxisScale = 'logY', 
sBkg_list: ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat', 'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT'] 
hBkg_integral_list (total 274188.85496806866): [249438.42506431916, 5778.916293955542, 11946.395296748857, 160.16806429978269, 88.86273862929193, 0.8854175968967408, 1488.6745258933422, 1043.3623537421488, 129.64939735444938, 2822.3673012315453, 1291.1485142976205] 
sBkg_list sorted by integral: [('QCD_0bCat', 249438.42506431916), ('QCD_2bCat', 11946.395296748857), ('QCD_1bCat', 5778.916293955542), ('WJetsToQQ_HT', 2822.3673012315453), ('TTToHadronic_powheg', 1488.6745258933422), ('ZJetsToQQ_HT', 1291.1485142976205), ('TTToSemiLeptonic_powheg', 1043.3623537421488), ('QCD_3bCat', 160.16806429978269), ('TTTo2L2Nu_powheg', 129.6493973544

/tmp/ssawant/ipykernel_27712/2615395917.py:519: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax[1].set_ylim(yRatioAxisRange_cal[0], yRatioAxisRange_cal[1])


...



 histo_name_toUse = 'hLeadingFatJet_nSubJets_bTag_L_SR', systematic = 'central', yAxisScale = 'linearY', 
sBkg_list: ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat', 'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT'] 
hBkg_integral_list (total 154996.62502522915): [6189.366150152652, 9139.137113502418, 120219.81631471704, 3723.6636911845503, 2407.8953133376112, 21.07977398862262, 6024.99116533005, 3877.827576830678, 243.72466236407675, 508.9132785345107, 2640.2099852868882] 
sBkg_list sorted by integral: [('QCD_2bCat', 120219.81631471704), ('QCD_1bCat', 9139.137113502418), ('QCD_0bCat', 6189.366150152652), ('TTToHadronic_powheg', 6024.99116533005), ('TTToSemiLeptonic_powheg', 3877.827576830678), ('QCD_3bCat', 3723.6636911845503), ('ZJetsToQQ_HT', 2640.2099852868882), ('QCD_4bCat', 2407.8953133376112), ('WJetsToQQ_HT', 508.9132785345107), ('TTTo2L2Nu_powheg', 243.72466236407675), ('QCD_5bAndMoreCat

/tmp/ssawant/ipykernel_27712/2615395917.py:519: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax[1].set_ylim(yRatioAxisRange_cal[0], yRatioAxisRange_cal[1])




 histo_name_toUse = 'hLeadingFatJet_nSV_SR', systematic = 'central', yAxisScale = 'logY', 
sBkg_list: ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat', 'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT'] 
hBkg_integral_list (total 154943.3200599151): [6189.366150152652, 9138.917123242023, 120186.57354988261, 3717.6287537675666, 2395.962268013536, 21.07977398862262, 6024.357545407602, 3877.2796230510803, 243.72466236407678, 508.9132785345108, 2639.5173315108004] 
sBkg_list sorted by integral: [('QCD_2bCat', 120186.57354988261), ('QCD_1bCat', 9138.917123242023), ('QCD_0bCat', 6189.366150152652), ('TTToHadronic_powheg', 6024.357545407602), ('TTToSemiLeptonic_powheg', 3877.2796230510803), ('QCD_3bCat', 3717.6287537675666), ('ZJetsToQQ_HT', 2639.5173315108004), ('QCD_4bCat', 2395.962268013536), ('WJetsToQQ_HT', 508.9132785345108), ('TTTo2L2Nu_powheg', 243.72466236407678), ('QCD_5bAndMoreCat', 21.07977398

/tmp/ssawant/ipykernel_27712/2615395917.py:519: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax[1].set_ylim(yRatioAxisRange_cal[0], yRatioAxisRange_cal[1])




 histo_name_toUse = 'hMET_pT_SR', systematic = 'central', yAxisScale = 'linearY', 
sBkg_list: ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat', 'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT'] 
hBkg_integral_list (total 154996.30601296842): [6189.366150152652, 9139.137113502416, 120219.81631471706, 3723.6636911845508, 2407.8953133376112, 21.079773988622623, 6024.99116533005, 3877.5085645699464, 243.72466236407678, 508.9132785345108, 2640.2099852868905] 
sBkg_list sorted by integral: [('QCD_2bCat', 120219.81631471706), ('QCD_1bCat', 9139.137113502416), ('QCD_0bCat', 6189.366150152652), ('TTToHadronic_powheg', 6024.99116533005), ('TTToSemiLeptonic_powheg', 3877.5085645699464), ('QCD_3bCat', 3723.6636911845508), ('ZJetsToQQ_HT', 2640.2099852868905), ('QCD_4bCat', 2407.8953133376112), ('WJetsToQQ_HT', 508.9132785345108), ('TTTo2L2Nu_powheg', 243.72466236407678), ('QCD_5bAndMoreCat', 21.07977398862262